In [37]:
from urllib.request import urlopen
from collections import OrderedDict
url = "https://sheets.googleapis.com/$discovery/rest?version=v4"

r = urlopen(url).read()
import json
j=json.loads(r.decode(), object_hook=OrderedDict)

def dump_l(l, indent=0):
    for item in l:
        if isinstance(item, list):
            dump_l(item, indent+1)
        elif isinstance(item, dict):
            dump_d(item, indent+1)
        else:
            print(indent*" ", item, sep="")
        
def dump_d(d, indent=0):
    for k, v in d.items():
        if isinstance(v, dict):
            print(indent*" ", k, ":", sep="")
            dump_d(v, indent+1)
        elif isinstance(v, list):
            dump_l(v, indent+1)
        else:
            assert isinstance(v, (int, str)), type(v)
            print(indent*" ", k, ': ', v, sep="")
            
def dump_resources(d, indent=0):
    for k, v in d.items():
        if k == 'resources':
            for k2, v2 in v.items():
                print(indent*" ", k2, ":", sep="")
                dump_resources(v2, indent+1)

In [38]:
dump_resources(j)

spreadsheets:
 sheets:
 values:


In [1]:
from goffice3.sheets import load_creds
from goffice3.sheets import GSession

In [2]:
class GClient():
    """ The client layer performs backend requests using 
    the given session object. Client implements the v4 api. 
    The client is defined as a hierarchy of collections in 
    the same way as the actual google api, to make 
    implementation directly reflect the google api structure. """
    def __init__(self, session):
        self.session = session
        self.spreadsheets = GCSpreadsheets(session)
        
    @classmethod
    def from_creds(cls, creds):
        return cls(GSession(creds))
    
    
class GCSpreadsheets():
    def __init__(self, session):
        self.session = session
        self.sheets = GCSheets(session)
        self.values = GCValues(session)
        
    def batchUpdate(self, ssid, updates):
        raise NotImplementedError
        
    def create(self):
        raise NotImplementedError
        
    def get(self):
        raise NotImplementedError
        
        
class GCValues():
    def __init__(self, session):
        self.session = session

    def batchGet(self, ssid):
        raise NotImplementedError
    
    def batchUpdate(self, ssid):
        raise NotImplementedError
        
    def get(self, ssid, ss_range):
        raise NotImplementedError
    
    def update(self, ssid, ss_range):
        raise NotImplementedError
        
        
class GCSheets():
    def __init__(self, session):
        self.session = session
    
    def copyTo(self):
        raise NotImplementedError

In [3]:
import os
os.chdir(os.path.expanduser("~/Documents/pyprojects/google_auths/.auth"))
c = GClient(GSession(load_creds('_exp_auth_key.json')))